# Deploying Model 

In [ ]:
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()

sess = sagemaker.Session()
from sagemaker import image_uris


training_image = image_uris.retrieve(
    region = sess.boto_region_name, framework = "object-detection", version = "1"
)
print(training_image)

Creating endpoint and model deployment

In [ ]:
model = sagemaker.model.Model(
image_uri = training_image,
model_data = 'S3_URI_best model path',
role = role    
)

In [ ]:
endpoint_name = ''

deployment = model.deploy(          #deploying model in Jupyterlab AWS
initial_instance_count = 1,
instance_type = "ml.m4.xlarge",  # no gpu require
endpoint_name = endpoint_name)

Visualizing the inferences

In [ ]:
import json 
import boto3
runtime = boto3.client(service_name = "runtime.sagemaker")

In [ ]:
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg#pyplot


def visualize_detection(img_file, dets, thresh = 0.6):
    img = mpimg.imread(img_file)
    plt.imshow(img)
    width = img.shape[1]
    height = img.shape[0]
    colors = dict()
    num_detections = 0
    for det in dets:
        (klass,score,x0,y0,x1,y1) = det #0
        if score < thresh:
            continue
        num_detections +=1
        cls_id = int(klass)#1
        if cls_id not in colors:
            colors[cls_id] = (random.random(),random.random(),random.random()) #(0.1,0.5,0.4)
        xmin = int(x0*width)
        ymin = int(y0*height)
        xmax = int(x1*width)
        ymax = int(y1*height) 
        
        
        rect = plt.Rectangle(
            (xmin,ymin),
            xmax - xmin,
            ymax - ymin,
            fill = False,
            edgecolor = colors[cls_id],
            linewidth = 3.5
        )
        plt.gca().add_patch(rect)
        plt.gca().text(
        xmin,
        ymin-2,
        "{:.3f}".format(score),
        bbox = dict(facecolor = colors[cls_id], alpha = 0.5),
        fontsize = 12,
        color = "white"
        )
    print("Number of detections" + str(num_detections))
    plt.show()
                

Invoking Endpoint

In [ ]:
def show_plastic_prediction(filename, ep, thresh = 0.3):
    b = ""
    with open(filename, "rb" ) as image:
        f = image.read()
        b = bytearray(f)
    endpoint_response = runtime.invoke_endpoint(EndpointName = ep, ContentType = "image/jpeg", Body = b)
    results = endpoint_response["Body"].read()
    detections = json.loads(results)
    
    
    visualize_detection(filename, detections['prediction'], thresh)
    
    